In [ ]:
# **Photos To Monet**

In [ ]:
!pip install deepspeed

In [ ]:
import glob
import os
import shutil
import deepspeed as ds
import matplotlib.pyplot as plt
import numpy as np
import pytorch_lightning as L
import torch
import torch.nn.functional as F
import torchvision.transforms as T
from pytorch_lightning.utilities import CombinedLoader
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision.io import read_image
from torchvision.utils import make_grid, save_image
_ = L.seed_everything(0, workers=True)

# Data Preprocessing 

In [ ]:
def show_img(img_tensor, nrow, title=""):
    img_tensor = img_tensor.detach().cpu() * 0.5 + 0.5
    img_grid = make_grid(img_tensor, nrow=nrow).permute(1, 2, 0)
    plt.figure(figsize=(10, 7))
    plt.imshow(img_grid)
    plt.axis("off")
    plt.title(title)
    plt.show()

**Augmenting the images.**

Before loading the datasets, we define CustomTransform for image augmentation. This improves learning by introducing more variety in the images during training instead of learning from the same set of images, especially when we only have 300 Monet paintings.

In [ ]:
class CustomTransform(object):
    def __init__(self, load_dim=286, target_dim=256):
        self.transform_train = T.Compose([
            T.Resize((load_dim, load_dim), antialias=True),
            T.RandomCrop((target_dim, target_dim)),
            T.RandomHorizontalFlip(p=0.5),
            T.ColorJitter(brightness=0.2, contrast=0.2,
                          saturation=0.2, hue=0.1),
        ])
        self.transform = T.Resize((target_dim, target_dim), antialias=True)   
    def __call__(self, img, stage):
        if stage == "fit":
            img = self.transform_train(img)
        else:
            img = self.transform(img)
        return img * 2 - 1

**Storing dataset**

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, filenames, transform, stage):
        self.filenames = filenames
        self.transform = transform
        self.stage = stage
        
    def __len__(self):
        return len(self.filenames)
    
    def __getitem__(self, idx):
        img_name = self.filenames[idx]
        img = read_image(img_name) / 255.0
        return self.transform(img, stage=self.stage)

**Iterating through the datasets**

To prepare the datasets, we load them into DataLoader separately, which can then iterate through the datasets as needed. Because the training dataset contains both the Monet paintings and photos, we pass both dataloaders into CombinedLoader for training. 

In [ ]:
DEBUG = False

DM_CONFIG = {    
    "monet_dir": os.path.join("/kaggle/input/gan-getting-started/monet_jpg", "*.jpg"),
    "photo_dir": os.path.join("/kaggle/input/gan-getting-started/photo_jpg", "*.jpg"),
    
    "loader_config": {
        "num_workers": os.cpu_count(),
        "pin_memory": torch.cuda.is_available(),
    },
    "sample_size": 5,
    "batch_size": 1 if not DEBUG else 1,
}

In [ ]:
class CustomDataModule(L.LightningDataModule):
    def __init__(
        self,
        monet_dir,
        photo_dir, 
        loader_config,
        sample_size,
        batch_size,
    ):
        super().__init__()
        self.loader_config = loader_config
        self.sample_size = sample_size
        self.batch_size = batch_size       
            
        # store file paths
        self.monet_filenames = sorted(glob.glob(monet_dir))
        self.photo_filenames = sorted(glob.glob(photo_dir))
        
        # define transformations for image augmentation
        self.transform = CustomTransform()
        
    def setup(self, stage):
        if stage == "fit":
            self.train_monet = CustomDataset(self.monet_filenames, self.transform, stage)
            self.train_photo = CustomDataset(self.photo_filenames, self.transform, stage)
            
        if stage in ["fit", "test", "predict"]:
            # to be used for test and prediction datasets as well
            self.valid_photo = CustomDataset(self.photo_filenames, self.transform, None)
            
    def train_dataloader(self):
        loader_config = {
            "shuffle": True,
            "drop_last": True,
            "batch_size": self.batch_size,
            **self.loader_config,
        }
        loader_monet = DataLoader(self.train_monet, **loader_config)
        loader_photo = DataLoader(self.train_photo, **loader_config)
        loaders = {"monet": loader_monet, "photo": loader_photo}
        return CombinedLoader(loaders, mode="max_size_cycle")
    
    def val_dataloader(self):
        return DataLoader(
            self.valid_photo,
            batch_size=self.sample_size,
            **self.loader_config,
        )
    
    def test_dataloader(self):
        return self.val_dataloader()
    
    def predict_dataloader(self):
        return DataLoader(
            self.valid_photo,
            batch_size=self.batch_size,
            **self.loader_config,
        )

We check that the datamodule defined is working as intended by visualizing samples of the images below.

In [ ]:
dm_sample = CustomDataModule(batch_size=5, **{k: v for k, v in DM_CONFIG.items() if k != "batch_size"})
dm_sample.setup("fit")
train_loader = dm_sample.train_dataloader()
imgs = next(iter(train_loader))
show_img(imgs["monet"], nrow=5, title="Augmented Monet Paintings")
show_img(imgs["photo"], nrow=5, title="Augmented Photos")

# Building GAN Architecture

**U-Net Generator**

A common architecture for the CycleGAN generator is the U-Net. U-Net is a network which consists of a sequence of downsampling blocks followed by a sequence of upsampling blocks, giving it the U-shaped architecture. In the upsampling path, we concatenate the outputs of the upsampling blocks and the outputs of the downsampling blocks symmetrically. This can be seen as a kind of skip connection, facilitating information flow in deep networks and reducing the impact of vanishing gradients.

**ResNet generator**

Similar to the U-Net architecture, the ResNet generator consists of the downsampling path and upsampling path. The difference is that the ResNet generator does not have the long skip connections from the concatenations of outputs. Instead, the ResNet generator uses residual blocks between the two paths. These residual blocks have short skip connections where the original input is added to the output.

**Downsampling blocks**

The downsampling blocks use convolution layers to increase the number of feature maps while reducing the dimensions of the 2D image.

In [ ]:
class Downsampling(nn.Module):
    def __init__(
        self,
        in_channels, # 輸入通道數
        out_channels, # 輸出通道數
        kernel_size=4, # mask為4*4矩陣
        stride=2, # mask在進行卷積時的移動步幅
        padding=1, #mask在卷積時會讓圖片外框補像素(輸入輸出的大小會變一樣)
        norm=True, #true則進行歸一化
        lrelu=True,#true則使用激活函數LeakyReLU 否則使用ReLU
    ):
        super().__init__()
        self.block = nn.Sequential( #名為nn.Sequential的容器 內有nn.Conv2d一個2D卷積層
            nn.Conv2d(in_channels, out_channels,
                      kernel_size=kernel_size, stride=stride, padding=padding, bias=not norm),
        )
        if norm:
            self.block.append(nn.InstanceNorm2d(out_channels, affine=True))
        if lrelu is not None:
            self.block.append(nn.LeakyReLU(0.2, True) if lrelu else nn.ReLU(True))
        
    def forward(self, x): #輸入x給self.block返回self.block(x)
        return self.block(x)

**Upsampling blocks**

On the other hand, the upsampling blocks contain transposed convolution layers, which combine the learned features to output an image with the original size.

In [ ]:
class Upsampling(nn.Module): #上採樣:用插值的方式將特徵圖分辨率增加
    def __init__(
        self,
        in_channels,
        out_channels,
        kernel_size=4,
        stride=2,
        padding=1,
        output_padding=0,
        dropout=False, #drop層:防止過度擬合(訓練中選擇一部份神經元輸出為零)
    ):
        super().__init__()
        self.block = nn.Sequential( #執行上採樣
            nn.ConvTranspose2d(in_channels, out_channels,
                               kernel_size=kernel_size, stride=stride, 
                               padding=padding, output_padding=output_padding, bias=False),
            nn.InstanceNorm2d(out_channels, affine=True), #神經網路輸入標準化
        )
        if dropout: #如dropout為true
            self.block.append(nn.Dropout(0.5)) #則添加dropout(以0.5的機率讓神經元輸出為零)
        self.block.append(nn.ReLU(True)) #必定加上一個ReLU
        
    def forward(self, x):
        return self.block(x)

**Residual blocks**

As described above, the residual blocks have convolution layers where the original input is added to the output.

In [ ]:
class ResBlock(nn.Module): #用於增加網路深度與提高特徵表達的能力
    def __init__(self, in_channels, kernel_size=3, padding=1):
        super().__init__()
        self.block = nn.Sequential(
            nn.ReflectionPad2d(padding), #圖片外框補像素
            Downsampling(in_channels, in_channels, #下採樣 進行卷積可提取特徵
                         kernel_size=kernel_size, stride=1, padding=0, lrelu=False),
            nn.ReflectionPad2d(padding),
            Downsampling(in_channels, in_channels,
                         kernel_size=kernel_size, stride=1, padding=0, lrelu=None),
        )
        
    def forward(self, x):
        return x + self.block(x) #加上ResBlock輸出的值 減輕梯度消失的問題

**Building the generator**

With the building blocks defined, we can now build our CycleGAN generator. For reference, the output size of each block is commented below.

In [ ]:
class UNetGenerator(nn.Module):
    def __init__(self, hid_channels, in_channels, out_channels):
        super().__init__()
        self.downsampling_path = nn.Sequential(
            Downsampling(in_channels, hid_channels, norm=False), # 64x128x128 通道數x圖片大小
            Downsampling(hid_channels, hid_channels*2), # 128x64x64 將通道數翻倍並用卷積將大小減小
            Downsampling(hid_channels*2, hid_channels*4), # 256x32x32
            Downsampling(hid_channels*4, hid_channels*8), # 512x16x16
            Downsampling(hid_channels*8, hid_channels*8), # 512x8x8
            Downsampling(hid_channels*8, hid_channels*8), # 512x4x4
            Downsampling(hid_channels*8, hid_channels*8), # 512x2x2
            Downsampling(hid_channels*8, hid_channels*8, norm=False), # 512x1x1, instance norm does not work on 1x1
        ) # 將圖片用下採樣變成1*1大小
        self.upsampling_path = nn.Sequential(
            Upsampling(hid_channels*8, hid_channels*8, dropout=True), # (512+512)x2x2
            Upsampling(hid_channels*16, hid_channels*8, dropout=True), # (512+512)x4x4
            Upsampling(hid_channels*16, hid_channels*8, dropout=True), # (512+512)x8x8
            Upsampling(hid_channels*16, hid_channels*8), # (512+512)x16x16
            Upsampling(hid_channels*16, hid_channels*4), # (256+256)x32x32
            Upsampling(hid_channels*8, hid_channels*2), # (128+128)x64x64
            Upsampling(hid_channels*4, hid_channels), # (64+64)x128x128
        ) #將1*1圖片變成目標大小
        self.feature_block = nn.Sequential( #用卷積讓格式變成想要的
            nn.ConvTranspose2d(hid_channels*2, out_channels,
                               kernel_size=4, stride=2, padding=1), # 3x256x256
            nn.Tanh(), #確保像素值在-1到1之間 輸出為3x256x256(三原色所以是3)
        )
        
    def forward(self, x):
        skips = [] #儲存下採樣每一層的輸出
        for down in self.downsampling_path:
            x = down(x)
            skips.append(x)
        skips = reversed(skips[:-1]) #將skip倒轉並去掉最後一個元素 給予上採樣執行

        for up, skip in zip(self.upsampling_path, skips):
            x = up(x)
            x = torch.cat([x, skip], dim=1)
        return self.feature_block(x) #將上採樣特徵圖x和下採樣特徵圖skip進行融合 有助細節補充
    
class ResNetGenerator(nn.Module):
    def __init__(self, hid_channels, in_channels, out_channels, num_resblocks):
        super().__init__()
        self.model = nn.Sequential(
            nn.ReflectionPad2d(3), #反射填充圖像外框
            Downsampling(in_channels, hid_channels, #進行下採樣減小圖像並增加通道
                         kernel_size=7, stride=1, padding=0, lrelu=False), # 64x256x256
            Downsampling(hid_channels, hid_channels*2, kernel_size=3, lrelu=False), # 128x128x128
            Downsampling(hid_channels*2, hid_channels*4, kernel_size=3, lrelu=False), # 256x64x64
            *[ResBlock(hid_channels*4) for _ in range(num_resblocks)], # 256x64x64 建立多個ResBlock改善圖像品質
            Upsampling(hid_channels*4, hid_channels*2, kernel_size=3, output_padding=1), # 128x128x128
            Upsampling(hid_channels*2, hid_channels, kernel_size=3, output_padding=1), # 64x256x256
            nn.ReflectionPad2d(3),
            nn.Conv2d(hid_channels, out_channels, kernel_size=7, stride=1, padding=0), # 3x256x256 卷積轉換成最後輸出圖像
            nn.Tanh(),
        )
        
    def forward(self, x):
        return self.model(x)
    
def get_gen(gen_name, hid_channels, num_resblocks, in_channels=3, out_channels=3):
    if gen_name == "unet":
        return UNetGenerator(hid_channels, in_channels, out_channels)
    elif gen_name == "resnet":
        return ResNetGenerator(hid_channels, in_channels, out_channels, num_resblocks)
    else:
        raise NotImplementedError(f"Generator name '{gen_name}' not recognized.") #根據輸入選擇要用哪個生成器

**Patch Gan Generator**

Unlike conventional networks that output a single probability of the input image being real or fake, CycleGAN uses the PatchGAN discriminator that outputs a matrix of values. Intuitively, each value of the output matrix checks the corresponding portion of the input image. Values closer to 1 indicate real classification and values closer to 0 indicate fake classification.

**Building the discriminator**

In general, the PatchGAN discriminator consists of a sequence of convolution layers, which can be built using the downsampling blocks defined earlier.

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, hid_channels, in_channels=3):
        super().__init__()
        self.block = nn.Sequential(
            Downsampling(in_channels, hid_channels, norm=False), # 64x128x128
            Downsampling(hid_channels, hid_channels*2), # 128x64x64
            Downsampling(hid_channels*2, hid_channels*4), # 256x32x32
            Downsampling(hid_channels*4, hid_channels*8, stride=1), # 512x31x31 增加通道並減小圖像
            nn.Conv2d(hid_channels*8, 1, kernel_size=4, padding=1), # 1x30x30 用卷積輸出真或是假
        )
        
    def forward(self, x):
        return self.block(x)

**Image buffer**

The original CycleGAN implementation updates the discriminator using a history of generated images instead of the latest images generated. This is done by setting up an image buffer that stores previously generated images. With probability 0.5, each newly generated image is swapped with a previously generated image stored in the buffer. This stabilizes training by giving the discriminator access to past information.

In [ ]:
class ImageBuffer(object): #保持訓練穩定性:可以儲存之前生成的圖像 有心圖像時有50%機率進行代替
    def __init__(self, buffer_size):
        self.buffer_size = buffer_size
        if self.buffer_size > 0:
            # the current capacity of the buffer
            self.curr_cap = 0
            # initialize buffer as empty list
            self.buffer = []
    
    def __call__(self, imgs):
        # the buffer is not used
        if self.buffer_size == 0:
            return imgs
        
        return_imgs = []
        for img in imgs:
            img = img.unsqueeze(dim=0)
            
            # 緩衝區圖像還沒滿的話
            if self.curr_cap < self.buffer_size:
                self.curr_cap += 1
                self.buffer.append(img)
                return_imgs.append(img)
            else:
                p = np.random.uniform(low=0., high=1.)
                
                # swap images between input and buffer with probability 0.5
                if p > 0.5:
                    idx = np.random.randint(low=0, high=self.buffer_size)
                    tmp = self.buffer[idx].clone()
                    self.buffer[idx] = img
                    return_imgs.append(tmp)
                else:
                    return_imgs.append(img)
        return torch.cat(return_imgs, dim=0)

**CycleGan**

With the generator and discriminator architectures defined, we can now build CycleGAN, which consists of two generators and two discriminators:

* Generator for photo-to-Monet translation (gen_PM).
* Generator for Monet-to-photo translation (gen_MP).
* Discriminator for Monet paintings (disc_M).
* Discriminator for photos (disc_P).
We use the Adam optimizer for model training. To optimize the parameters, we need to define the loss functions:

* **Discriminator loss**:- For real images fed into the discriminator, the output matrix is compared against a matrix of 1s using the mean squared error. For fake images, the output matrix is compared against a matrix of 0s. This suggests that to minimize loss, the perfect discriminator outputs a matrix of 1s for real images and a matrix of 0s for fake images.
* **Generator loss:-** This is composed of three different loss functions below.
     *          *Adversarial loss* :-Fake images are fed into the discriminator and the output matrix is compared against a matrix of 1s using the mean squared error. To minimize loss, the generator needs to 'fool' the discriminator into thinking that the fake images are real and output a matrix of 1s.
    *         *Identity loss*:- When a Monet painting is fed into the photo-to-Monet generator, we should get back the same Monet painting because nothing needs to be transformed. The same applies for photos fed into the Monet-to-photo generator. To encourage identity mapping, the difference in pixel values between the input image and generated image is measured using the l1 loss.
    *         *Cycle loss* When a Monet painting is fed into the Monet-to-photo generator, and the generated image is fed back into the photo-to-Monet generator, it should transform back into the original Monet painting. The same applies for photos passed to the two generators to get back the original photos. To preserve information throughout this cycle, the l1 loss is used to measure the difference between the original image and the reconstructed image.

**Building the CycleGAN model**

In [ ]:
MODEL_CONFIG = { #模型配置設定
    # the type of generator, and the number of residual blocks if ResNet generator is used
    "gen_name": "unet", # types: 'unet', 'resnet'
    "num_resblocks": 6,
    # the number of filters in the first layer for the generators and discriminators
    "hid_channels": 64,
    # using DeepSpeed's FusedAdam (currently GPU only) is slightly faster
    "optimizer": ds.ops.adam.FusedAdam if torch.cuda.is_available() else torch.optim.Adam,
    # the learning rate and beta parameters for the Adam optimizer
    "lr": 2e-4,
    "betas": (0.5, 0.999),
    # the weights used in the identity loss and cycle loss
    "lambda_idt": 0.5,
    "lambda_cycle": (10, 10), # (MPM direction, PMP direction)
    # the size of the buffer that stores previously generated images
    "buffer_size": 100,
    # the number of epochs for training
    "num_epochs": 18 if not DEBUG else 2,
    # the number of epochs before starting the learning rate decay
    "decay_epochs": 18 if not DEBUG else 1,
}

In [ ]:
class CycleGAN(L.LightningModule):
    def __init__(
        self,
        gen_name,
        num_resblocks,
        hid_channels,
        optimizer,
        lr,
        betas,
        lambda_idt,
        lambda_cycle,
        buffer_size,
        num_epochs,
        decay_epochs,
    ):
        super().__init__()
        self.save_hyperparameters(ignore=["optimizer"])
        self.optimizer = optimizer
        self.automatic_optimization = False
        
        # define generators and discriminators
        self.gen_PM = get_gen(gen_name, hid_channels, num_resblocks)
        self.gen_MP = get_gen(gen_name, hid_channels, num_resblocks)
        self.disc_M = Discriminator(hid_channels)
        self.disc_P = Discriminator(hid_channels)
        
        # initialize buffers to store fake images
        self.buffer_fake_M = ImageBuffer(buffer_size)
        self.buffer_fake_P = ImageBuffer(buffer_size)
        
    def forward(self, img):
        return self.gen_PM(img)   #forward用輸入圖像生成新圖像
            
    def init_weights(self): #初始化模型權重
        def init_fn(m):
            if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.InstanceNorm2d)):
                nn.init.normal_(m.weight, 0.0, 0.02)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0.0)
        
        for net in [self.gen_PM, self.gen_MP, self.disc_M, self.disc_P]:
            net.apply(init_fn)
        
    def setup(self, stage):
        if stage == "fit":
            self.init_weights()
            print("Model initialized.") #初始化後顯示提示句子
            
    def get_lr_scheduler(self, optimizer): #學習速度調節器
        def lr_lambda(epoch):
            len_decay_phase = self.hparams.num_epochs - self.hparams.decay_epochs + 1.0
            curr_decay_step = max(0, epoch - self.hparams.decay_epochs + 1.0)
            val = 1.0 - curr_decay_step / len_decay_phase
            return max(0.0, val)
        
        return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)
    
    def configure_optimizers(self): #優化器和學習率調度器
        opt_config = {
            "lr": self.hparams.lr,
            "betas": self.hparams.betas,
        }
        opt_gen = self.optimizer(
            list(self.gen_PM.parameters()) + list(self.gen_MP.parameters()),
            **opt_config,
        )
        opt_disc = self.optimizer(
            list(self.disc_M.parameters()) + list(self.disc_P.parameters()),
            **opt_config,
        )
        optimizers = [opt_gen, opt_disc]
        schedulers = [self.get_lr_scheduler(opt) for opt in optimizers]
        return optimizers, schedulers
        
    def adv_criterion(self, y_hat, y): #使用均方誤差生成圖像與真實的差距
        return F.mse_loss(y_hat, y)
    
    def recon_criterion(self, y_hat, y): #使用絕對值誤差生成圖像與真實的差距
        return F.l1_loss(y_hat, y)
    
    def get_adv_loss(self, fake, disc): #fake為假的disc判別器
        fake_hat = disc(fake) #判別器判別假圖的評分 0到1 越像真越大
        real_labels = torch.ones_like(fake_hat) #把所有假圖標籤設為1
        adv_loss = self.adv_criterion(fake_hat, real_labels) #計算上面兩個的對抗損失
        return adv_loss #因為希望假圖越像真越好 判別器給出的機率最好為1 因此把所有假圖標籤設為1設為最佳狀態
    
    def get_idt_loss(self, real, idt, lambda_cycle): #real為真實的idt為生成的lambda_cycle為一參數
        idt_loss = self.recon_criterion(idt, real) #計算身份損失
        return self.hparams.lambda_idt * lambda_cycle * idt_loss
    
    def get_cycle_loss(self, real, recon, lambda_cycle): #real為真實的recon為循環一圈的
        cycle_loss = self.recon_criterion(recon, real) #計算循環一圈與真實的損失
        return lambda_cycle * cycle_loss
    
    def get_gen_loss(self):
        # calculate adversarial loss
        adv_loss_PM = self.get_adv_loss(self.fake_M, self.disc_M) #假莫內圖和判別器M的對抗損失
        adv_loss_MP = self.get_adv_loss(self.fake_P, self.disc_P) #假風景照和判別器P的對抗損失
        total_adv_loss = adv_loss_PM + adv_loss_MP #對抗損失和 表示生成器生成的圖像在鑑別器上被評估為真實圖像的程度
        
        # calculate identity loss
        lambda_cycle = self.hparams.lambda_cycle
        idt_loss_MM = self.get_idt_loss(self.real_M, self.idt_M, lambda_cycle[0])#莫內圖生成器將真莫內圖轉成假莫內圖的身份損失
        idt_loss_PP = self.get_idt_loss(self.real_P, self.idt_P, lambda_cycle[1])#風景照生成器將真實風景照轉成假風景照的身份損失
        total_idt_loss = idt_loss_MM + idt_loss_PP #身份損失之和 表示生成器在兩個方向上保持原始圖像內容的總損失
        
        # calculate cycle loss
        cycle_loss_MPM = self.get_cycle_loss(self.real_M, self.recon_M, lambda_cycle[0]) #真莫內圖循環一圈與原圖的損失
        cycle_loss_PMP = self.get_cycle_loss(self.real_P, self.recon_P, lambda_cycle[1]) #真風景圖循環一圈與原圖的損失
        total_cycle_loss = cycle_loss_MPM + cycle_loss_PMP #循環一致性損失之和，表示生成器生成的圖像在兩個方向上的轉換一致性
        
        # combine losses
        gen_loss = total_adv_loss + total_idt_loss + total_cycle_loss
        return gen_loss #全部損失
    
    def get_disc_loss(self, real, fake, disc): #判別器的損失
        # calculate loss on real images
        real_hat = disc(real) #判別器對真圖的判別機率
        real_labels = torch.ones_like(real_hat) #真圖都標上1
        real_loss = self.adv_criterion(real_hat, real_labels) #計算判別器對於真圖的對抗損失(真判別為真的程度)
        
        # calculate loss on fake images
        fake_hat = disc(fake.detach()) #假圖給判別器判別的結果(不影響生成器梯度)
        fake_labels = torch.zeros_like(fake_hat) #假圖都標上0
        fake_loss = self.adv_criterion(fake_hat, fake_labels) #計算判別器對於假圖的對抗損失 目標是讓鑑別器能夠正確地識別假圖像
        
        # combine losses
        disc_loss = (fake_loss + real_loss) * 0.5
        return disc_loss #判別器總損失
    
    def get_disc_loss_M(self):
        fake_M = self.buffer_fake_M(self.fake_M)
        return self.get_disc_loss(self.real_M, fake_M, self.disc_M) #真與假莫內圖計算判別器M的損失
    
    def get_disc_loss_P(self):
        fake_P = self.buffer_fake_P(self.fake_P)
        return self.get_disc_loss(self.real_P, fake_P, self.disc_P) #真與假風景照計算判別器P的損失
    
    def training_step(self, batch, batch_idx): #訓練步驟
        self.real_M = batch["monet"] #獲取真實風景照與真實莫內圖
        self.real_P = batch["photo"] 
        opt_gen, opt_disc = self.optimizers() #獲取鑑別器與生成器的優化器

        # generate fake images
        self.fake_M = self.gen_PM(self.real_P) #生成器生成假莫內圖
        self.fake_P = self.gen_MP(self.real_M) #生成器生成假風景照
        
        # generate identity images
        self.idt_M = self.gen_PM(self.real_M) #莫內圖生成器將真莫內圖轉成假莫內圖的身份損失
        self.idt_P = self.gen_MP(self.real_P) #風景照生成器將真風景照轉成假風景照的身份損失
        
        # reconstruct images
        self.recon_M = self.gen_PM(self.fake_P) #莫內圖生成器將假風景照轉成假莫內圖的循環損失
        self.recon_P = self.gen_MP(self.fake_M) #風景照生成器將假莫內圖轉成假風景照的循環損失
    
        # train generators
        self.toggle_optimizer(opt_gen) #切換到生成器的優化器
        gen_loss = self.get_gen_loss() #計算生成器的總損失(含對抗損失、循環一致性損失、身份損失，要最小化)
        opt_gen.zero_grad() #梯度歸零
        self.manual_backward(gen_loss) #根據損失 計算生成器梯度
        opt_gen.step() #使用生成優化器更新生成器參數 根據梯度進行更新
        self.untoggle_optimizer(opt_gen) #切換到判別器的優化器
        
        # train discriminators
        self.toggle_optimizer(opt_disc) #切換到判別器的優化器 
        disc_loss_M = self.get_disc_loss_M() #兩個判別器的損失
        disc_loss_P = self.get_disc_loss_P()
        opt_disc.zero_grad() #梯度清零
        self.manual_backward(disc_loss_M) #下面和上面一樣
        self.manual_backward(disc_loss_P)
        opt_disc.step()
        self.untoggle_optimizer(opt_disc)
        
        # 紀錄訓練時的損失
        metrics = {
            "gen_loss": gen_loss,
            "disc_loss_M": disc_loss_M,
            "disc_loss_P": disc_loss_P,
        }
        self.log_dict(metrics, on_step=False, on_epoch=True, prog_bar=True)
        
    def validation_step(self, batch, batch_idx): #評估模型在驗證數據集上的性能
        self.display_results(batch, batch_idx, "validate")
    
    def test_step(self, batch, batch_idx): #評估模型在測試數據集上的性能
        self.display_results(batch, batch_idx, "test")
        
    def predict_step(self, batch, batch_idx): #對新數據進行預測
        return self(batch)
    
    def display_results(self, batch, batch_idx, stage): #展示用程式(不是很需要理解)
        real_P = batch
        fake_M = self(real_P)
        
        if stage == "validate":
            title = f"Epoch {self.current_epoch+1}: Photo-to-Monet Translation"
        else:
            title = f"Sample {batch_idx+1}: Photo-to-Monet Translation"

        show_img(
            torch.cat([real_P, fake_M], dim=0),
            nrow=len(real_P),
            title=title,
        )
    
    def on_train_epoch_start(self): #記錄當前的學習率
        # record learning rates
        curr_lr = self.lr_schedulers()[0].get_last_lr()[0]
        self.log("lr", curr_lr, on_step=False, on_epoch=True, prog_bar=True)
        
    def on_train_epoch_end(self):
        # 更新學習率
        for sch in self.lr_schedulers():
            sch.step()
        
        # 印出當前訓練周期的狀態
        logged_values = self.trainer.progress_bar_metrics
        print(
            f"Epoch {self.current_epoch+1}",
            *[f"{k}: {v:.5f}" for k, v in logged_values.items()],
            sep=" - ",
        )
        
    def on_train_end(self):
        print("Training ended.") #印出一條消息表示訓練已經結束
        
    def on_predict_epoch_end(self): #在模型的預測過程中（例如，使用測試數據進行預測）的每個預測訓練周期結束時被呼叫
        predictions = self.trainer.predict_loop.predictions #取得預測結果、計算生成的圖像的總數、印出生成的圖像數量
        num_batches = len(predictions)
        batch_size = predictions[0].shape[0]
        last_batch_diff = batch_size - predictions[-1].shape[0]
        print(f"Number of images generated: {num_batches*batch_size-last_batch_diff}.")

# Model Training 

In [ ]:
TRAIN_CONFIG = {  #各種訓練參數的設定
    "accelerator": "gpu" if torch.cuda.is_available() else "cpu",
    
    # train on 16-bit precision
    "precision": "16-mixed" if torch.cuda.is_available() else 32,
    
    # train on single GPU
    "devices": 1,
    
    # save checkpoint only for last epoch by default
    "enable_checkpointing": True,
    
    # disable logging for simplicity
    "logger": False,
    
    # the number of epochs for training (we limit the number of train/predict batches during debugging)
    "max_epochs": MODEL_CONFIG["num_epochs"],
    "limit_train_batches": 1.0 if not DEBUG else 2,
    "limit_predict_batches": 1.0 if not DEBUG else 5,
    
    # the maximum amount of time for training, in case we exceed run-time of 5 hours
    "max_time": {"hours": 4, "minutes": 55},
    
    # use a small subset of photos for validation/testing (we limit here for flexibility)
    "limit_val_batches": 1,
    "limit_test_batches": 5,
    
    # disable sanity check before starting the training routine
    "num_sanity_val_steps": 0,
    
    # the frequency to visualize the progress of adding Monet style
    "check_val_every_n_epoch": 6 if not DEBUG else 1,
}

In [ ]:
dm = CustomDataModule(**DM_CONFIG)
model = CycleGAN(**MODEL_CONFIG)
trainer = L.Trainer(**TRAIN_CONFIG)
trainer.fit(model, datamodule=dm)

**Testing the model on other sample photos**

In [ ]:
_ = trainer.test(model, datamodule=dm)

# Submission

Computing the predictions can be done by running the predict method to generate the Monet-style images given the input photos.

In [ ]:
predictions = trainer.predict(model, datamodule=dm)

**Saving the generated images**

In [ ]:
os.makedirs("../images", exist_ok=True)
idx = 0
for tensor in predictions:
    for monet in tensor:
        save_image(
            monet.float().squeeze() * 0.5 + 0.5, 
            fp=f"../images/{idx}.jpg",
        )
        idx += 1

shutil.make_archive("/kaggle/working/images", "zip", "/kaggle/images")